In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from datetime import datetime
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline

In [9]:
online_action = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', engine='python', encoding='utf-8-sig')
transaction = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', engine='python', encoding='utf-8-sig')
customer = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv', engine='python', encoding='utf-8-sig')
production = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv', engine='python', encoding='utf-8-sig')

In [109]:
result=pd.read_excel('쇼핑 성향에 따른 혜택 선호도 조사(응답) (1).xlsx', encoding='utf-8-sig')

In [111]:
신중도 = {
    '1시간 이상' : 'A',
    '10분 이내' : 'B',
    '10분 ~ 1시간' : 'C',
    '평균 10회 이상' : 'C',
    '평균 3 ~ 9회' : 'A',
    '거의 누르지 않는다' : 'B',
    '평균 1 ~ 2회' : 'A',
    '검색어를 다양하게 바꿔가면서 검색해보는 스타일 ex) 패딩, 남자패딩, 노스페이스 패딩' : 'A',
    '하나의 검색어만 주로 사용하는 스타일 ex) 패딩, 우유, 옥수수' : 'B',
    '검색어를 사용하지 않은 스타일' : 'C'
}

In [112]:
습관도 = {
    '아니오' : 'A, C',
    '예' : 'B, D',
    '저녁 (6:00 ~ 12:00)' : 'C, D',
    '평균 10회 이상' : 'D',
    '새벽 (12:00 ~ 6:00)' : 'C, D',
    '오후 (12:00 ~ 6:00)' : 'C, D',
    '오전 (6:00 ~ 12:00)' : 'C, D',
    '주로 쇼핑하는 시간대가 정해져있지 않음' : 'A, B',
    '자주 방문하는 쇼핑몰이 없음' : 'A',
    '평균 1회 ~ 2회' : 'B',
    '평균 3 ~ 9회' : 'C'
}

In [113]:
충성도 = {
    '종종 있다' : 'C',
    '없다' : 'B',
    '자주 있다' : 'A',
    '자주 방문하는 쇼핑몰이 없음' : 'A',
    '평균 10회 이상' : 'D',
    '평균 1회 ~ 2회' : 'B',
    '평균 3 ~ 9회' : 'C',
    '0 ~ 20만원' : 'A',
    '20만원 ~ 50만원' : 'C',
    '50만원 이상' : 'B' 
}

In [114]:
result['구매경과시간'] = result['구매경과시간'].map(lambda x: 신중도[x])
result['Back_nums'] = result['Back_nums'].map(lambda x: 신중도[x])
result['검색어유사도'] = result['검색어유사도'].map(lambda x: 신중도[x])

In [115]:
result['반복구매'] = result['반복구매'].map(lambda x: 습관도[x])
result['시간집중도'] = result['시간집중도'].map(lambda x: 습관도[x])
result['관심도_습관'] = result['관심도_습관'].map(lambda x: 습관도[x])

In [116]:
result['관심도_충성'] = result['관심도_충성'].map(lambda x: 충성도[x])
result['코드검색'] = result['코드검색'].map(lambda x: 충성도[x])
result['구매금액'] = result['구매금액'].map(lambda x: 충성도[x])

In [127]:
result

,구매경과시간,Back_nums,검색어유사도,반복구매,시간집중도,관심도_습관,관심도_충성,코드검색,구매금액,혜택1,혜택2,혜택3,혜택4,혜택5,성별,나이,신중도,습관도,충성도
0,A,C,A,"A, C","C, D",A,A,C,A,4,5,2,1,2,남,20대,0,0,0
1,A,C,B,"A, C","C, D",D,D,B,A,3,4,1,2,5,여,20대,0,0,0
2,B,B,B,"A, C","A, B",A,A,B,A,2,1,3,4,5,남,20대,0,0,0
3,C,C,A,"B, D","C, D",B,B,C,A,1,3,5,2,4,남,20대,0,0,0
4,C,A,B,"B, D","C, D",B,B,B,A,1,3,4,2,5,남,20대,0,0,0
5,C,C,A,"B, D","C, D",B,B,C,C,1,2,4,3,5,남,30대,0,0,0
6,C,C,A,"A, C","C, D",B,B,B,C,5,1,3,2,4,여,20대,0,0,0
7,C,A,A,"A, C","A, B",D,D,B,C,2,4,3,1,5,남,30대,0,0,0
8,C,A,B,"A, C","C, D",A,A,C,A,1,2,4,3,5,여,20대,0,0,0
9,B,B,A,"B, D","C, D",B,B,C,A,1,4,3,2,5,여,20대,0,0,0


In [118]:
result.혜택1=result.혜택1.apply(lambda x : x[0]).astype(int)
result.혜택2=result.혜택2.apply(lambda x : x[0]).astype(int)
result.혜택3=result.혜택3.apply(lambda x : x[0]).astype(int)
result.혜택4=result.혜택4.apply(lambda x : x[0]).astype(int)
result.혜택5=result.혜택5.apply(lambda x : x[0]).astype(int)

In [119]:
result['신중도'] = 0
result['습관도'] = 0
result['충성도'] = 0

In [108]:
result.to_csv('혜택구하기.csv')

In [155]:
clustering=pd.read_csv('설문지_사람_성격분류.csv', engine='python', encoding='utf-8-sig')

In [162]:
clustering.drop('사람', axis=1, inplace=True)

In [163]:
result[['신중도', '습관도', '충성도']] = clustering

In [165]:
신중도=result.pivot_table(index = '신중도', values = ['혜택1', '혜택2', '혜택3', '혜택4', '혜택5'], aggfunc = 'sum')
습관도=result.pivot_table(index = '습관도', values = ['혜택1', '혜택2', '혜택3', '혜택4', '혜택5'], aggfunc = 'sum')
충성도=result.pivot_table(index = '충성도', values = ['혜택1', '혜택2', '혜택3', '혜택4', '혜택5'], aggfunc = 'sum')

In [166]:
신중도.loc['신중도_A']=신중도.loc['신중도_A']/신중도.loc['신중도_A'].sum()
신중도.loc['신중도_B']=신중도.loc['신중도_B']/신중도.loc['신중도_B'].sum()
신중도.loc['신중도_C']=신중도.loc['신중도_C']/신중도.loc['신중도_C'].sum()

In [171]:
습관도.loc['습관도_A']=습관도.loc['습관도_A']/습관도.loc['습관도_A'].sum()
습관도.loc['습관도_B']=습관도.loc['습관도_B']/습관도.loc['습관도_B'].sum()
습관도.loc['습관도_C']=습관도.loc['습관도_C']/습관도.loc['습관도_C'].sum()
습관도.loc['습관도_D']=습관도.loc['습관도_D']/습관도.loc['습관도_D'].sum()

In [172]:
충성도.loc['충성도_A']=충성도.loc['충성도_A']/충성도.loc['충성도_A'].sum()
충성도.loc['충성도_B']=충성도.loc['충성도_B']/충성도.loc['충성도_B'].sum()
충성도.loc['충성도_C']=충성도.loc['충성도_C']/충성도.loc['충성도_C'].sum()

In [173]:
신중도.to_csv('신중도.csv')
습관도.to_csv('습관도.csv')
충성도.to_csv('충성도.csv')

In [174]:
신중도

,혜택1,혜택2,혜택3,혜택4,혜택5
신중도,,,,,
신중도_A,0.130217,0.252087,0.210351,0.136895,0.270451
신중도_B,0.112281,0.207018,0.228070,0.168421,0.284211
신중도_C,0.105431,0.242812,0.191693,0.162939,0.297125


In [175]:
습관도

,혜택1,혜택2,혜택3,혜택4,혜택5
습관도,,,,,
습관도_A,0.114719,0.242424,0.188312,0.168831,0.285714
습관도_B,0.114286,0.231746,0.222222,0.142857,0.288889
습관도_C,0.130556,0.250000,0.216667,0.136111,0.266667
습관도_D,0.116667,0.183333,0.266667,0.150000,0.283333


In [176]:
충성도

,혜택1,혜택2,혜택3,혜택4,혜택5
충성도,,,,,
충성도_A,0.117845,0.235690,0.178451,0.178451,0.289562
충성도_B,0.117333,0.234667,0.224000,0.138667,0.285333
충성도_C,0.133333,0.266667,0.200000,0.160000,0.240000


In [178]:
신중도.loc['신중도_A'] + 습관도.loc['습관도_B'] + 충성도.loc['충성도_C']

혜택1    0.377836
혜택2    0.750500
혜택3    0.632573
혜택4    0.439752
혜택5    0.799340
dtype: float64